# Hull and White

In [25]:
import numpy as np
import pandas as pd
from math import ceil

In [26]:
r = 0.0541
a = -.562
s = 1
t = 0
T = 0
vol = .23
deltat = .1
optionType = "C"

In [27]:
def futureDiscountFunction(r, a, s, t, T, vol, deltat):
    """
    # Hull & White closed form solution
    Inputs
    r =      # interest rate at time t
    s =      # Maturity
    a =      # reversion parameter
    T =      # expiry
    deltat = # deltat for derivative
    vol =    # volatility
    
    Output
    # Price of zerocoupon bond at time T with maturity s
    """
    
    # market prices of T year and s year discount bonds
    ptT = np.exp(-r*T)
    pts = np.exp(-r*s)

    B = (1/a)*(1-np.exp(-a*(s-T)))
    slope = (np.log(np.exp(-r*(T+deltat))) - np.log(np.exp(-r*(T-deltat))))/(2*deltat)
    lnA = (np.log(pts/ptT)) - (B*slope) - ((1/(4*(a**3)))*(vol**2)*((np.exp(-a*(s-t))-np.exp(-a*(T-t)))**2)*(np.exp(2*a*(T-t))-1))

    pTs = np.exp(lnA)*(np.exp(-B*r))

    return(pTs)
futureDiscountFunction(r, a, s, t, T, vol, deltat)

0.9473373680281515

### Trinomial Tree Construction for Hull & White

In [4]:
# full procedure to construct the trees
# assume i > 0 and and that theta((i-1)dt), Qi-1,j, Mui-1,j, Pi-1,j, Ri-1,j, di-1,j
# have all been found for all j at time step i-1(r0,0 = R(1), Q0,0 = 1)

# Step 1 initialize parameters
alpha = 0.1
sigma = 0.014
N = 2
T = 2

#tmp
K = 1

# step 2 pre-compute constants
dt = T/N
dr = sigma*np.sqrt(3*dt)

# step 3 initialize yield curve
initial_yield_curve = [0.05, 0.055, 0.06, 0.0625, 0.065]
R = initial_yield_curve
P = [0]*(N+1)

for i in range(N + 1):
    R[i] = initial_yield_curve[i]
    P[i] = np.exp(-R[i] * i * dt)

# step 3.1 define arrays for node value storage: rows = 2*N + 1, Columns = N + 1
r = np.zeros((N+1, 2*N + 1))
d = np.zeros((N+1, 2*N + 1))   
Q = np.zeros((N+1, 2*N + 1))
Theta = np.zeros(N+1) # one theta value per i
Mu = np.zeros((N+1, 2*N + 1))
Eta = np.zeros((N+1, 2*N + 1))
Pup = np.zeros((N+1, 2*N + 1))
Pmid = np.zeros((N+1, 2*N + 1))   
Pdown = np.zeros((N+1, 2*N + 1))

# step 4 Initialize first node
r[0][0] = R[0]
Q[0][0] = 1
d[0][0] = np.exp(-r[0][0] * dt)
Theta[0] = ((2*R[1])/dt) + (((sigma**2)*dt) / 2) + (((alpha*r[0][0]*(dt**2)) - (2*r[0][0]*dt))/dt**2)
Mu[0][0] = (Theta[0]-(alpha*r[0][0]))*dt
Eta[0][0] = Mu[0,0] + (1-K)*dr
Pup[0][0] = (((((sigma**2)*dt)+(Eta[0][0]**2))/(2*(dr**2))) + (Eta[0][0]/(2*dr)))
Pmid[0][0] = 1- ((((sigma**2)*dt)+(Eta[0][0]**2))/((dr**2)))
Pdown[0][0] = 1 - Pup[0][0] - Pmid[0][0]

# step 4 Evolve tree for the short rate
for i in range(N+1):
    
    low_node = -i
    top_node = i
    Q1 = 0
    
    if i > 0:
        # evolving r and d
        for j in range(low_node, (top_node + 1)):
            ji = j + i

            r[i][ji] = r[0][0] + (j * dr)
            d[i][ji] = np.exp(-r[i][j] * dt)

            # updating pure security prices
            if low_node < j < top_node:
                Q[i][ji-1] += Q[i-1][ji-1]*Pdown[i-1][ji-1]*d[i-1][ji-1]
                Q[i][ji] += Q[i-1][ji-1]*Pmid[i-1][ji-1]*d[i-1][ji-1]
                Q[i][ji+1] += Q[i-1][ji-1]*Pup[i-1][ji-1]*d[i-1][ji-1]
        
        # Theta calculation
        for j in range(low_node, top_node + 1):
            ji = j + i
            Q1 = Q1 + Q[i][ji] * np.exp((-2*r[i][ji]*dt)+(alpha*r[i][ji]*dt*dt))
            
        
        Theta[i] = ((1/dt)*(i+2)*R[i+1]) + (((sigma**2)*dt)/2) + ((1/(dt*dt))*np.log(Q1))
        
        # calculate drift and decide branch process
        for j in range(low_node, top_node + 1):
            ji = j + i
            Mu[i][ji] = (Theta[i] - alpha*r[i][ji])*dt

            # decide branch process {Determines K}
        
            # calculate probabiliites
            Eta[i][ji] = Mu[i][ji] + (j-K)*dr
            Pup[i][ji] = (((((sigma**2)*dt)+(Eta[i][ji]**2))/(2*(dr**2))) + (Eta[i][ji]/(2*dr)))
            Pmid[i][ji] = 1- ((((sigma**2)*dt)+(Eta[i][ji]**2))/((dr**2)))
            Pdown[i][ji] = 1 - Pup[i][ji] - Pmid[i][ji]
            
print("r", r)
print("Q", Q)
print("Mu", Mu)
print("Pup", Pup)
print("Pmid", Pmid)
print("Pdown", Pdown)

r [[0.05       0.         0.         0.         0.        ]
 [0.02575129 0.05       0.07424871 0.         0.        ]
 [0.00150258 0.02575129 0.05       0.07424871 0.09849742]]
Q [[1.         0.         0.         0.         0.        ]
 [0.04295587 0.46919294 0.43908062 0.         0.        ]
 [0.08223859 0.20427954 0.21559341 0.24225132 0.17352428]]
Mu [[0.010098   0.         0.         0.         0.        ]
 [0.01369161 0.01126674 0.00884187 0.         0.        ]
 [0.05983915 0.05741428 0.05498941 0.05256454 0.05013967]]
Pup [[0.46159276 0.         0.         0.         0.        ]
 [0.47912276 0.04229209 0.41546142 0.         0.        ]
 [0.0421875  0.41814014 1.60409278 3.60004542 6.40599805]]
Pmid [[ 0.49324897  0.          0.          0.          0.        ]
 [-1.39361294  0.3800484   0.53370974  0.          0.        ]
 [ 0.38335016  0.53144488 -0.9404604  -4.03236568 -8.74427096]]
Pdown [[0.04515826 0.         0.         0.         0.        ]
 [1.91449018 0.57765951 0.0508